<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2022-09-10 17:04:38--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2022-09-10 17:04:38--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2022-09-10 17:04:39--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['joni-mitchell.txt',
 'bruno-mars.txt',
 'disney.txt',
 'bob-dylan.txt',
 'radiohead.txt',
 'adele.txt',
 'janisjoplin.txt',
 'r-kelly.txt',
 'ludacris.txt',
 'patti-smith.txt',
 'jimi-hendrix.txt',
 'lil-wayne.txt',
 'leonard-cohen.txt',
 'kanye-west.txt',
 'alicia-keys.txt',
 'cake.txt',
 'nicki-minaj.txt',
 'drake.txt',
 'dickinson.txt',
 'kanye.txt',
 'rihanna.txt',
 'bob-marley.txt',
 'bieber.txt',
 'lady-gaga.txt',
 'dr-seuss.txt',
 'missy-elliott.txt',
 'paul-simon.txt',
 'notorious_big.txt',
 'bjork.txt',
 'al-green.txt',
 'Kanye_West.txt',
 'lin-manuel-miranda.txt',
 'johnny-cash.txt',
 'beatles.txt',
 'Lil_Wayne.txt',
 'nirvana.txt',
 'blink-182.txt',
 'amy-winehouse.txt',
 'michael-jackson.txt',
 'nursery_rhymes.txt',
 'notorious-big.txt',
 'bruce-springsteen.txt',
 'lorde.txt',
 'prince.txt',
 'eminem.txt',
 'dolly-parton.txt',
 'nickelback.txt',
 'dj-khaled.txt',
 'britney-spears.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 116364.7734375
Loss after epoch 1: 66042.9296875
Loss after epoch 2: 65961.796875
Loss after epoch 3: 65764.8125
Loss after epoch 4: 64249.375
Loss after epoch 5: 64699.375
Loss after epoch 6: 64866.1875
Loss after epoch 7: 66130.125
Loss after epoch 8: 64368.625
Loss after epoch 9: 62594.9375
Loss after epoch 10: 62067.875
Loss after epoch 11: 61172.9375
Loss after epoch 12: 59934.125
Loss after epoch 13: 58675.1875
Loss after epoch 14: 58000.5
Loss after epoch 15: 57134.9375
Loss after epoch 16: 52772.75
Loss after epoch 17: 51840.25
Loss after epoch 18: 51582.625
Loss after epoch 19: 51034.375


(156986, 287740)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9320977926254272),
 ('sleep', 0.8989176154136658),
 ('try', 0.8954020738601685),
 ('cry', 0.885643720626831),
 ('seems', 0.8802488446235657),
 ('twist', 0.8787717223167419),
 ('peace', 0.8779224753379822),
 ('years', 0.8730202913284302),
 ('high', 0.8667396903038025),
 ('saw', 0.8658844232559204)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('four', -0.2464623898267746),
 ('shake', -0.25986626744270325),
 ('our', -0.2613790035247803),
 ('day', -0.2639576494693756),
 ('sun', -0.26874369382858276),
 ('five', -0.27138620615005493),
 ('comes', -0.27201294898986816),
 ('here', -0.27436816692352295),
 ('goo', -0.2858683466911316),
 ('yellow', -0.28905290365219116)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9878801107406616),
 ('three', 0.9863857626914978),
 ('six', 0.9778979420661926),
 ('two', 0.9678113460540771),
 ('seven', 0.9490659832954407),
 ('sixty', 0.8907109498977661),
 ('one', 0.8399722576141357),
 ('crying', 0.8322727680206299),
 ("i'm", 0.8235415816307068),
 ('strawberry', 0.8131641149520874)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9544175863265991),
 ('buy', 0.9503788352012634),
 ('much', 0.9210140705108643),
 ('just', 0.8962178826332092),
 ('hide', 0.880431056022644)]

In [18]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [19]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [20]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
